In [2]:
from dotenv import load_dotenv
from langchain_ollama import ChatOllama

load_dotenv()

True

In [3]:
from langchain.agents import AgentState

class CustomState(AgentState):
    favourite_colour: str

## Write to state

In [5]:
from langchain.tools import tool, ToolRuntime
from langgraph.types import Command
from langchain.messages import ToolMessage

@tool
def update_favourite_colour(favourite_colour: str, runtime: ToolRuntime) -> Command:
    """Update the favourite colour of the user in the state once they've revealed it."""
    return Command(update={
        "favourite_colour": favourite_colour, 
        "messages": [ToolMessage("Successfully updated favourite colour", tool_call_id=runtime.tool_call_id)]}
        )

In [6]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver
from langchain_ollama import ChatOllama

model = ChatOllama(model="lfm2.5-thinking")

agent = create_agent(
    model=model,
    tools=[update_favourite_colour],
    checkpointer=InMemorySaver(),
    state_schema=CustomState
)

In [7]:
from langchain.messages import HumanMessage

response = agent.invoke(
    { "messages": [HumanMessage(content="My favourite colour is green")]},
    {"configurable": {"thread_id": "1"}}
)

In [8]:
from pprint import pprint

pprint(response)

{'favourite_colour': 'green',
 'messages': [HumanMessage(content='My favourite colour is green', additional_kwargs={}, response_metadata={}, id='9894f6c0-f55b-45c3-bb00-b15bde04dcff'),
              AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'lfm2.5-thinking', 'created_at': '2026-02-20T09:08:02.053504Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1177915833, 'load_duration': 41338166, 'prompt_eval_count': 100, 'prompt_eval_duration': 98639167, 'eval_count': 139, 'eval_duration': 1017357167, 'logprobs': None, 'model_name': 'lfm2.5-thinking', 'model_provider': 'ollama'}, id='lc_run--019c7a4e-58e8-76d1-a075-fc01a7b35150-0', tool_calls=[{'name': 'update_favourite_colour', 'args': {'favourite_colour': 'green'}, 'id': 'b0e1cbb3-e2b5-4b5a-93f0-af2d28b113e7', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 100, 'output_tokens': 139, 'total_tokens': 239}),
              ToolMessage(content='Successfully updated favourite col

In [9]:
response = agent.invoke(
    { 
        "messages": [HumanMessage(content="Hello, how are you?")],
        "favourite_colour": "green"
    },
    {"configurable": {"thread_id": "10"}}
)

pprint(response)

{'favourite_colour': 'green',
 'messages': [HumanMessage(content='Hello, how are you?', additional_kwargs={}, response_metadata={}, id='b6bd8908-fb18-49fb-83d7-a87882a902ff'),
              AIMessage(content="I don't have emotions, but I'm here to assist you! How can I help you today?", additional_kwargs={}, response_metadata={'model': 'lfm2.5-thinking', 'created_at': '2026-02-20T09:08:22.256867Z', 'done': True, 'done_reason': 'stop', 'total_duration': 7323970333, 'load_duration': 35361375, 'prompt_eval_count': 101, 'prompt_eval_duration': 62302584, 'eval_count': 955, 'eval_duration': 7105372924, 'logprobs': None, 'model_name': 'lfm2.5-thinking', 'model_provider': 'ollama'}, id='lc_run--019c7a4e-8fd3-7870-85c1-6e38d661b072-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 101, 'output_tokens': 955, 'total_tokens': 1056})]}


## Read state

In [11]:
@tool
def read_favourite_colour(runtime: ToolRuntime) -> str:
    """Read the favourite colour of the user from the state."""
    try:
        return runtime.state["favourite_colour"]
    except KeyError:
        return "No favourite colour found in state"

from langchain_ollama import ChatOllama

model = ChatOllama(model="lfm2.5-thinking")

agent = create_agent(
    model=model,
    tools=[update_favourite_colour, read_favourite_colour],
    checkpointer=InMemorySaver(),
    state_schema=CustomState
)

In [12]:
response = agent.invoke(
    { "messages": [HumanMessage(content="My favourite colour is green")]},
    {"configurable": {"thread_id": "1"}}
)

pprint(response)

{'favourite_colour': 'green',
 'messages': [HumanMessage(content='My favourite colour is green', additional_kwargs={}, response_metadata={}, id='a4d27579-d1a9-4903-b0ac-ad1f6a8aaa6d'),
              AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'lfm2.5-thinking', 'created_at': '2026-02-20T09:09:04.288516Z', 'done': True, 'done_reason': 'stop', 'total_duration': 4036184375, 'load_duration': 35116583, 'prompt_eval_count': 145, 'prompt_eval_duration': 79994166, 'eval_count': 515, 'eval_duration': 3853395804, 'logprobs': None, 'model_name': 'lfm2.5-thinking', 'model_provider': 'ollama'}, id='lc_run--019c7a4f-40da-7ae2-a830-c808f4bf48e1-0', tool_calls=[{'name': 'update_favourite_colour', 'args': {'favourite_colour': 'green'}, 'id': '5695cc50-5957-454d-a2f8-1ee851e5b5ec', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 145, 'output_tokens': 515, 'total_tokens': 660}),
              ToolMessage(content='Successfully updated favourite col

In [13]:
response = agent.invoke(
    { "messages": [HumanMessage(content="What's my favourite colour?")]},
    {"configurable": {"thread_id": "1"}}
)

pprint(response)

{'favourite_colour': 'green',
 'messages': [HumanMessage(content='My favourite colour is green', additional_kwargs={}, response_metadata={}, id='a4d27579-d1a9-4903-b0ac-ad1f6a8aaa6d'),
              AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'lfm2.5-thinking', 'created_at': '2026-02-20T09:09:04.288516Z', 'done': True, 'done_reason': 'stop', 'total_duration': 4036184375, 'load_duration': 35116583, 'prompt_eval_count': 145, 'prompt_eval_duration': 79994166, 'eval_count': 515, 'eval_duration': 3853395804, 'logprobs': None, 'model_name': 'lfm2.5-thinking', 'model_provider': 'ollama'}, id='lc_run--019c7a4f-40da-7ae2-a830-c808f4bf48e1-0', tool_calls=[{'name': 'update_favourite_colour', 'args': {'favourite_colour': 'green'}, 'id': '5695cc50-5957-454d-a2f8-1ee851e5b5ec', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 145, 'output_tokens': 515, 'total_tokens': 660}),
              ToolMessage(content='Successfully updated favourite col